In [365]:
from sql_functions import get_dataframe
import pandas as pd
import capstone_functions as cf
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from scipy import interpolate
from sklearn import linear_model
import seaborn as sns

pd.options.mode.chained_assignment = None  # default='warn'

# Variables

In [366]:
# How many mechanics are in the top mechanics
top_XX_mechanic = 25

#schema and table names, we are using in this notebook
schema = 'bgg_data'
main = 'ml_boardgame_stats'
subdomain = 'subdomain'
unique_subdomain = 'unique_subdomain'
kickstarter = 'kickstarter_unique_campaigns'
slug = 'unique_slug_bgg_id'
mechanic = 'mechanics'
unique_mechanics = 'unique_mechanics'
honor = 'honor_clean'

## Create Dataframes for Kickstarter and Mechanic and build df for the top XX mechanics

In [367]:

df_ks = get_dataframe(f"SELECT * FROM {schema}.{kickstarter}")
df_slug = get_dataframe(f"SELECT * FROM {schema}.{slug}")
df_mech = get_dataframe(f"SELECT * FROM {schema}.{mechanic}")
df_u_mech = get_dataframe(f"SELECT * FROM {schema}.{unique_mechanics}")
df_main = get_dataframe(f"SELECT * FROM {schema}.{main}")
df_sub = get_dataframe(f"SELECT * FROM {schema}.{subdomain}")
df_u_sub = get_dataframe(f"SELECT * FROM {schema}.{unique_subdomain}")

#df_marketplace = cf.avg_price_from_marketplace()
#df_honor = get_dataframe(f"SELECT * FROM {schema}.{honor}")

Merge kickstarter table with bgg_ids and rename column bgg_id to id

In [368]:
df_ks = pd.merge(df_slug,df_ks,on='slug')
df_ks.rename({'bgg_id':'id'},axis=1,inplace=True)

## Create a list with the top XX mechanics

create dataframe with all mechanics and merge it with the kickstarter dataframe

In [369]:
df_mech = pd.merge(df_mech,df_u_mech,on='mechanic_id')
df_mech = pd.merge(df_ks,df_mech,on='id')

In [370]:
top_mechanics_list = list(df_mech.groupby(['mechanic']).count().sort_values(ascending=False,axis=0,by='id').reset_index()["mechanic"].head(top_XX_mechanic))

#### Alternative for top XX mechanics calculation

In [371]:
# top mechanics by bgg rating
df_mech_02 = pd.merge(df_main,df_mech,on='id')

In [372]:
top_rated_mechanics_list = list(df_mech_02.groupby('mechanic').mean('average').sort_values('average',ascending=False).reset_index().mechanic.head(top_XX_mechanic))

### reduce the dataframe such that IDs are unique

choose between two different ways of defining "top mechanics"

In [373]:
#mechanics_list = top_mechanics_list
mechanics_list = top_rated_mechanics_list
print(mechanics_list)

df_mech["is_in_top_XX_mechanics"] = df_mech["mechanic"].isin(mechanics_list)
df_mech = df_mech[['id','is_in_top_XX_mechanics']]
df_mech = df_mech.groupby('id').sum().reset_index()

['predictive bid', 'automatic resource growth', 'pattern movement', 'line of sight', 'increase value of unchosen resources', 'impulse movement', 'hidden movement', 'zone of control', 're-rolling and locking', 'cube tower', 'follow', 'programmed movement', 'worker placement, different worker types', 'investment', 'roles with asymmetric information', 'deck construction', 'command cards', 'worker placement with dice workers', 'flicking', 'movement template', 'scenario / mission / campaign game', 'three dimensional movement', 'measurement movement', 'income', 'bribery']


Because there are multiple mechanics for one ID, it is possible that for the same ID there are some mechanics in the top and others are not.
- create new column with True/False if ID is in top XX categories (XX will be set in the top code field: "Variables" -> "top_XX_mechanic")

In [374]:
df_mech[f"top_{top_XX_mechanic}_mechanic"] = df_mech.is_in_top_XX_mechanics > 0

In [375]:
df_mech = df_mech[['id',f"top_{top_XX_mechanic}_mechanic"]]

- merge Kickstarter with subdomains => 1021 non-null

In [376]:
df_sub = pd.merge(df_sub,df_u_sub,on="subdomain_id")
df_ks_sub = pd.merge(df_ks,df_sub,on='id')

- merge Kickstarter_subdomains with main => ~330 entries

In [377]:
df_ks_sub_main = pd.merge(df_ks_sub,df_main,on='id')

- Build new columns with goal in USD

In [378]:
df_ks_sub_main["usd_goal"] = df_ks_sub_main['goal']*(df_ks_sub_main.pledged/df_ks_sub_main.usd_pledged)

merge with mechanics dataframe => 326 non-null entries

In [379]:
df_ks_sub_main_mech = pd.merge(df_ks_sub_main,df_mech,on='id')

## Machine Learning
- Extract only necessary columns for our ML and drop all null

In [380]:
df_ML = df_ks_sub_main_mech[['country','usd_pledged','subdomain_name','min_players','max_players','min_playtime','max_playtime','min_age','averageweight','usd_goal',f"top_{top_XX_mechanic}_mechanic"]]
df_ML.dropna(inplace=True);

In [381]:
df_ML[f"top_{top_XX_mechanic}_mechanic"] = df_ML[f"top_{top_XX_mechanic}_mechanic"].astype(int)

In [382]:
df_ML["min_playtime"].fillna(df_ML["max_playtime"].median(),inplace=True);
df_ML["max_playtime"].fillna(df_ML["max_playtime"].median(),inplace=True);


In [383]:
df_ML = df_ML[(np.abs(stats.zscore(df_ML['min_playtime'])) < 2.5)]
df_ML = df_ML[(np.abs(stats.zscore(df_ML['max_playtime'])) < 3)]

## Build dummies for subdomain and countries

In [384]:
subdomain_dummy = pd.get_dummies(df_ML.subdomain_name.apply(pd.Series).stack(), drop_first=True).groupby(level=0).sum()
subdomain_dummy.columns = subdomain_dummy.columns.str.strip()

In [385]:
country_dummy = pd.get_dummies(df_ML.country.apply(pd.Series).stack(), drop_first=True).groupby(level=0).sum()
country_dummy.columns = country_dummy.columns.str.strip()

In [386]:
df = pd.concat([df_ML,subdomain_dummy], axis=1)
df = df.drop(["subdomain_name"], axis=1)
#df.columns

In [387]:
df = pd.concat([df,country_dummy], axis=1)
df = df.drop(["country"], axis=1)
#df.columns

In [388]:
X = df[['DE', 'GB', 'US',
        "Children's", "Customizable", "Family", "Party", "Strategy", "Thematic", "Wargames", 
        "min_players", "min_playtime", "min_age", "averageweight", 'usd_goal',f"top_{top_XX_mechanic}_mechanic"]]

In [389]:
y = df.usd_pledged

#### Investigate the whole dataframe with all columns of interest

In [390]:
# create an OLS model
our_model = sm.OLS(y, X)

# use the data to calculate the intercept and slope
model_results = our_model.fit()

# return the output of the model
model_results.summary() # summary contains eg. 'const' (intercept) and 'slope' of the regression equation.

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            usd_pledged   R-squared (uncentered):                   0.384
Model:                            OLS   Adj. R-squared (uncentered):              0.351
Method:                 Least Squares   F-statistic:                              11.66
Date:                Fri, 15 Jul 2022   Prob (F-statistic):                    1.72e-23
Time:                        15:48:12   Log-Likelihood:                         -4353.8
No. Observations:                 315   AIC:                                      8740.
Df Residuals:                     299   BIC:                                      8800.
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
DE               3.544e+05   1.33e+05      2.663      0.008    9.25e+04    6.16e+05
GB               1326.1065   5.32e+04      0.025      0.980   -1.03e+05    1.06e+05
US               2.136e+04   3.63e+04      0.589      0.556      -5e+04    9.27e+04
Children's      -7.769e+04   1.03e+05     -0.753      0.452   -2.81e+05    1.25e+05
Customizable    -8.066e+04   2.56e+05     -0.315      0.753   -5.84e+05    4.23e+05
Family            1.31e+04   5.15e+04      0.254      0.800   -8.83e+04    1.14e+05
Party            4.153e+05    8.2e+04      5.062      0.000    2.54e+05    5.77e+05
Strategy        -9862.6789   6.03e+04     -0.164      0.870   -1.29e+05    1.09e+05
Thematic         5.522e+04   6.27e+04      0.880      0.379   -6.82e+04    1.79e+05
Wargames        -8.364e+04   5.98e+04     -1.399      0.163   -2.01e+05     3.4e+04
min_players     -2.363e+04   1.93e+04     -1.222      0.223   -6.17e+04    1.44e+04
min_playtime     -637.9149    500.461     -1.275      0.203   -1622.787     346.958
min_age         -2720.3233   4684.762     -0.581      0.562   -1.19e+04    6498.958
averageweight    8.066e+04   2.37e+04      3.399      0.001     3.4e+04    1.27e+05
usd_goal            0.1171      0.074      1.589      0.113      -0.028       0.262
top_25_mechanic  1.806e+05   4.87e+04      3.711      0.000    8.48e+04    2.76e+05
==============================================================================
Omnibus:                      278.352   Durbin-Watson:                   1.547
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6807.499
Skew:                           3.564   Prob(JB):                         0.00
Kurtosis:                      24.630   Cond. No.                     3.77e+06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 3.77e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Pick the most significant input parameters

In [391]:
params = ['averageweight', 
        'min_playtime', 
        'min_players', 
        'min_age', 
        f"top_{top_XX_mechanic}_mechanic", 
        'Party', 
        'Thematic', 
        "Children's", 
        'DE',
        'usd_goal']

# Für Jannik

In [392]:
X = df[params]

In [393]:
#X = sm.add_constant(X)
#X.describe()

In [394]:
# create an OLS model
our_model = sm.OLS(y, X)

# use the data to calculate the intercept and slope
model_results = our_model.fit()

# return the output of the model
model_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            usd_pledged   R-squared (uncentered):                   0.374
Model:                            OLS   Adj. R-squared (uncentered):              0.353
Method:                 Least Squares   F-statistic:                              18.19
Date:                Fri, 15 Jul 2022   Prob (F-statistic):                    5.21e-26
Time:                        15:48:13   Log-Likelihood:                         -4356.5
No. Observations:                 315   AIC:                                      8733.
Df Residuals:                     305   BIC:                                      8771.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
averageweight    8.185e+04   2.16e+04      3.793      0.000    3.94e+04    1.24e+05
min_playtime    -1025.0151    456.851     -2.244      0.026   -1923.993    -126.037
min_players     -1.456e+04   1.77e+04     -0.823      0.411   -4.93e+04    2.02e+04
min_age         -3303.7160   4369.513     -0.756      0.450   -1.19e+04    5294.492
top_25_mechanic  1.667e+05   4.81e+04      3.469      0.001    7.22e+04    2.61e+05
Party            4.207e+05   7.21e+04      5.836      0.000    2.79e+05    5.63e+05
Thematic         8.116e+04   4.13e+04      1.965      0.050    -127.306    1.62e+05
Children's      -7.205e+04    9.9e+04     -0.728      0.467   -2.67e+05    1.23e+05
DE                 3.3e+05   1.29e+05      2.561      0.011    7.65e+04    5.83e+05
usd_goal            0.1144      0.072      1.590      0.113      -0.027       0.256
==============================================================================
Omnibus:                      275.707   Durbin-Watson:                   1.548
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6611.640
Skew:                           3.519   Prob(JB):                         0.00
Kurtosis:                      24.312   Cond. No.                     1.94e+06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.94e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [395]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [396]:
params

['averageweight',
 'min_playtime',
 'min_players',
 'min_age',
 'top_25_mechanic',
 'Party',
 'Thematic',
 "Children's",
 'DE',
 'usd_goal']

In [397]:
test_params = [1,20,2,8,1,1,1,1,1,100000]
predicted_pledged_amount = regr.predict([test_params])
predicted_pledged_amount

/opt/homebrew/Caskroom/miniconda/base/envs/nf_sql/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([944635.73480226])

# Test - For upcoming games

In [398]:
main = 'unfiltered_main_stats_cleaned'
family_query = f"SELECT * FROM {schema}.family_bgg;"

In [399]:
df_main = get_dataframe(f"SELECT * FROM {schema}.{main}")
df_family = get_dataframe(family_query)

In [400]:
df_upcoming = pd.merge(df_main,df_family,on='id')

In [362]:
all_upcoming_games = df_upcoming.query("family_type == 'admin' and family_value == 'upcoming releases'")

In [115]:
all_upcoming_games.drop(["yearpublished", 'trading', 'numcomments','family_type', 'family_value',
       'family_id','average', 'user_rated'],axis=1,inplace=True)

In [116]:
all_upcoming_games.nunique()

id               5499
min_players         8
max_players        29
playtime           84
min_playtime       43
max_playtime       84
min_age            22
num_owned         399
wanting           106
wishing           346
numweights         32
averageweight      87
kickstarter         2
dtype: int64

In [117]:
all_upcoming_games.dropna(inplace=True)

In [118]:
all_upcoming_games.head(2)

,id,min_players,max_players,playtime,min_playtime,max_playtime,min_age,num_owned,wanting,wishing,numweights,averageweight,kickstarter
281,344240,3.0,5.0,45.0,20.0,45.0,9,18,0,1,1,1.0,True
314,344268,2.0,4.0,40.0,20.0,40.0,9,7,8,31,2,2.0,True


In [119]:
all_upcoming_games.min_players = all_upcoming_games.min_players.astype(int)
all_upcoming_games.max_players = all_upcoming_games.max_players.astype(int)
all_upcoming_games.min_playtime = all_upcoming_games.min_playtime.astype(int)
all_upcoming_games.max_playtime = all_upcoming_games.max_playtime.astype(int)

In [120]:
all_upcoming_games.drop_duplicates(inplace=True)
all_upcoming_games.dropna(inplace=True)

In [123]:
test = pd.merge(all_upcoming_games,df_mech,on='id')

In [124]:
test

,id,min_players,max_players,playtime,min_playtime,max_playtime,min_age,num_owned,wanting,wishing,numweights,averageweight,kickstarter,mechanic_id,mechanic
0,344240,3,5,45.0,20,45,9,18,0,1,1,1.0,True,2041,open drafting
1,344240,3,5,45.0,20,45,9,18,0,1,1,1.0,True,2020,simultaneous action selection
2,344240,3,5,45.0,20,45,9,18,0,1,1,1.0,True,2685,player elimination
3,344240,3,5,45.0,20,45,9,18,0,1,1,1.0,True,2003,rock-paper-scissors
4,344240,3,5,45.0,20,45,9,18,0,1,1,1.0,True,2686,take that
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,298086,1,6,120.0,45,120,10,11,5,57,2,2.5,True,2676,grid movement
6010,298086,1,6,120.0,45,120,10,11,5,57,2,2.5,True,2876,race
6011,298086,1,6,120.0,45,120,10,11,5,57,2,2.5,True,2897,variable set-up
6012,298086,1,6,120.0,45,120,10,11,5,57,2,2.5,True,2875,end game bonuses
